In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/09/15 18:26:24 WARN Utils: Your hostname, DESKTOP-5U9FK5P resolves to a loopback address: 127.0.1.1; using 172.26.185.11 instead (on interface eth0)
22/09/15 18:26:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 18:26:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sdf_transactions1 = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot')
sdf_transactions2 = spark.read.parquet('../data/tables/transactions_20210828_20220227_snapshot')
sdf_transactions3 = spark.read.parquet('../data/tables/transactions_20220228_20220828_snapshot')

sdf_transactions = sdf_transactions1.union(sdf_transactions2)
sdf_transactions = sdf_transactions.union(sdf_transactions3)

In [3]:
sdf_transactions.show(10, truncate=False)

+-------+------------+------------------+------------------------------------+--------------+
|user_id|merchant_abn|dollar_value      |order_id                            |order_datetime|
+-------+------------+------------------+------------------------------------+--------------+
|18478  |62191208634 |63.255848959735246|949a63c8-29f7-4ab0-ada4-99ac50a88952|2021-08-20    |
|2      |15549624934 |130.3505283105634 |6a84c3cf-612a-4574-835b-144a47353eff|2021-08-20    |
|18479  |64403598239 |120.15860593212783|b10dcc33-e53f-4254-863c-de5266810cbc|2021-08-20    |
|3      |60956456424 |136.6785200286976 |0f09c5a5-784e-4477-b049-8ee4dd069b7b|2021-08-20    |
|18479  |94493496784 |72.96316578355305 |f6c78c1a-4600-4c5f-8e97-6e9eb534b586|2021-08-20    |
|3      |76819856970 |448.529684285612  |5ace6a24-cdf0-4aa3-b571-1d9406b352b5|2021-08-20    |
|18479  |67609108741 |86.4040605836911  |d0e180f0-cb06-42a3-bd1a-c47dca15bc55|2021-08-20    |
|3      |34096466752 |301.5793450525113 |6fb1ff48-24bb-4f97-

In [4]:
sdf_transactions.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [23]:
def validateABN(merchant_abn):

    str_abn = str(merchant_abn)
    i = 0
    sum = 0

    for digit in str_abn:

        digit = int(digit)
        
        if i == 0:
            digit -= 1
            sum += digit * 10
        elif i == 1:
            sum += digit
        elif i == 2:
            sum += digit * 3
        elif i == 3:
            sum += digit * 5
        elif i == 4:
            sum += digit * 7
        elif i == 5:
            sum += digit * 9
        elif i == 6:
            sum += digit * 11
        elif i == 7:
            sum += digit * 13
        elif i == 8:
            sum += digit * 15
        elif i == 9:
            sum += digit * 17
        elif i == 10:
            sum += digit * 19

        i += 1
    
    if sum % 89 == 0:
        return True
    else:
        return False

In [24]:
validateABN(51824753556)

True

22/09/08 15:23:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 526676 ms exceeds timeout 120000 ms
22/09/08 15:23:54 WARN SparkContext: Killing executors is not supported by current scheduler.


In [5]:
sdf_consumer = spark.read.csv('../data/tables/tbl_consumer.csv', sep='|', header=True)

In [42]:
# Seperating sdf by state
from pyspark.sql import functions as F

VIC_sdf = sdf_consumer.where(
    (F.col('state') == 'VIC' ))

NSW_sdf = sdf_consumer.where(
    (F.col('state') == 'NSW' ))

QLD_sdf = sdf_consumer.where(
    (F.col('state') == 'QLD' ))

WA_sdf = sdf_consumer.where(
    (F.col('state') == 'WA' ))

TAS_sdf = sdf_consumer.where(
    (F.col('state') == 'TAS' ))

SA_sdf = sdf_consumer.where(
    (F.col('state') == 'SA' ))

ACT_sdf = sdf_consumer.where(
    (F.col('state') == 'ACT' ))

NT_sdf = sdf_consumer.where(
    (F.col('state') == 'NT' ))

In [19]:
VIC_sdf.count()

117525

In [23]:
VIC_sdf.orderBy("postcode", ascending=0)

name,address,state,postcode,gender,consumer_id
Cody Casey,71856 Jones Park ...,VIC,8785,Male,105717
Blake Phillips,56069 Moore Isle ...,VIC,8785,Male,1425156
Patricia Cervantes,6724 Benton Alley...,VIC,8785,Undisclosed,468647
Sarah Shah,71321 Roberts Wall,VIC,8785,Female,16233
Brenda Browning,091 Scott Stream ...,VIC,8785,Female,126110
Tina Phillips,41992 Stanley Run...,VIC,8785,Female,1070577
Victoria Harris,5166 Michael Exte...,VIC,8785,Female,1162087
Lauren Gray,6556 Cameron Shores,VIC,8785,Female,583532
Wayne Munoz,3525 Robert Forges,VIC,8785,Male,1373154
Willie Bell II,929 Anderson Spur...,VIC,8785,Male,1494155


In [22]:
VIC_sdf= VIC_sdf.where(
    ((F.col('postcode') >=  '3000') & (F.col('postcode') <= '3999'))
    | ((F.col('postcode') >=  '8000') & (F.col('postcode') <= '8999'))
)
VIC_sdf.count()

117376

In [43]:
NT_sdf.count()

7764

In [13]:
NT_sdf.show(50)

+--------------------+--------------------+-----+--------+-----------+-----------+
|                name|             address|state|postcode|     gender|consumer_id|
+--------------------+--------------------+-----+--------+-----------+-----------+
|       Jill Jones MD|  40693 Henry Greens|   NT|     862|     Female|    1194530|
|Dr. Allison Bryan MD|   993 Collins Ports|   NT|     885|     Female|    1092515|
|      Kristen Watson| 35081 Rowland Plaza|   NT|     851|     Female|     654114|
|       Shannon Horne|457 Holmes Trail ...|   NT|     907|     Female|     702865|
|        George Davis| 417 Jackson Viaduct|   NT|     822|       Male|     928836|
|       Austin Turner|  83006 Simpson Keys|   NT|     871|       Male|     351368|
|     Matthew Johnson|3667 Ortiz Plains...|   NT|     834|       Male|     704784|
|          Diana Wall|4006 Buchanan Pla...|   NT|     906|     Female|     145411|
|         Faith Chase|     2565 Ian Radial|   NT|     813|     Female|    1027922|
|   

In [44]:
NT_sdf= NT_sdf.where(
    (F.col('postcode') >=  '800') & (F.col('postcode') <= '999'))
NT_sdf.count()

7764

In [15]:
TAS_sdf.count()

18878

In [16]:
TAS_sdf= TAS_sdf.where(
    (F.col('postcode') >=  '7000') & (F.col('postcode') <= '7999'))
TAS_sdf.count()

18878

In [24]:
WA_sdf.count()

79146

In [32]:
WA_sdf= WA_sdf.where(
    ((F.col('postcode') >=  '6000') & (F.col('postcode') <= '6797'))
    | ((F.col('postcode') >=  '6800') & (F.col('postcode') <= '6999'))
)
WA_sdf.count()

78837

In [33]:
SA_sdf.count()

54973

In [36]:
SA_sdf= SA_sdf.where(
    (F.col('postcode') >=  '5000') & (F.col('postcode') <= '5999'))
SA_sdf.count()

54973

In [37]:
QLD_sdf.count()

72861

In [38]:
QLD_sdf= QLD_sdf.where(
    ((F.col('postcode') >=  '4000') & (F.col('postcode') <= '4999'))
    | ((F.col('postcode') >=  '9000') & (F.col('postcode') <= '9999'))
)
QLD_sdf.count()

72861

In [40]:
NSW_sdf.count()

144188

In [41]:
NSW_sdf= NSW_sdf.where(
    ((F.col('postcode') >=  '1000') & (F.col('postcode') <= '2599'))
    | ((F.col('postcode') >=  '2619') & (F.col('postcode') <= '2899'))
    | ((F.col('postcode') >=  '2921') & (F.col('postcode') <= '2999'))
)
NSW_sdf.count()

143580

In [45]:
ACT_sdf.count()

4664

In [46]:
ACT_sdf= ACT_sdf.where(
    ((F.col('postcode') >=  '200') & (F.col('postcode') <= '300'))
    | ((F.col('postcode') >=  '2600') & (F.col('postcode') <= '2618'))
    | ((F.col('postcode') >=  '2900') & (F.col('postcode') <= '2920'))
)
ACT_sdf.count()

4664